In [7]:
import pandas as pd 

# import streamlit as st
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from keras.models import Sequential,load_model
# import streamlit as st
min_max_scaler = preprocessing.MinMaxScaler()


## RUL 
'Data Set: FD001
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: ONE (HPC Degradation)

Data Set: FD002
Train trjectories: 260
Test trajectories: 259
Conditions: SIX 
Fault Modes: ONE (HPC Degradation)

Data Set: FD003
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Data Set: FD004
Train trjectories: 248
Test trajectories: 249
Conditions: SIX 
Fault Modes: TWO (HPC Degradation, Fan Degradation)



Experimental Scenario

Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine – i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. The data is contaminated with sensor noise.

The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The objective of the competition is to predict the number of remaining operational cycles before failure in the test set, i.e., the number of operational cycles after the last cycle that the engine will continue to operate. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:
1)	unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  26


Reference: A. Saxena, K. Goebel, D. Simon, and N. Eklund, “Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation”, in the Proceedings of the Ist International Conference on Prognostics and Health Management (PHM08), Denver CO, Oct 2008.



In [10]:
sequence_length = 50
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)
cols_normalize=['id','cycle','RUL','label1','label2']
test_df = pd.read_csv('./Dataset/CMAPSSData/test_FD004.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
cols_normalize = test_df.columns.difference(['id', 'cycle', 'RUL', 'label1', 'label2'])

norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[cols_normalize]),
                            columns=cols_normalize,
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns=test_df.columns)
test_df = test_df.reset_index(drop=True)


In [11]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.000000,0.476271,0.831354,1.0,0.626985,0.656561,0.654027,0.501557,0.507937,...,0.452491,0.996016,0.574006,0.362211,0.0,0.631579,0.864693,1.0,0.478320,0.483780
1,0.0,0.002062,0.595087,0.736342,0.0,0.238089,0.007372,0.041486,0.016265,0.293184,...,0.086570,0.000968,0.039366,0.945325,0.0,0.042105,0.000000,0.0,0.141260,0.137380
2,0.0,0.004124,0.999810,1.000000,1.0,0.000000,0.127258,0.270191,0.200168,0.000000,...,0.004212,0.996487,0.637839,0.423747,0.0,0.273684,0.627907,1.0,0.006775,0.016466
3,0.0,0.006186,0.999893,0.997862,1.0,0.000000,0.126889,0.269020,0.203579,0.000000,...,0.002988,0.996348,0.660202,0.410736,0.0,0.273684,0.627907,1.0,0.012534,0.005168
4,0.0,0.008247,0.833363,0.997625,1.0,0.060269,0.181349,0.305358,0.229632,0.146592,...,0.131165,0.996265,0.599510,0.390686,0.0,0.326316,0.651163,1.0,0.157859,0.157613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41209,1.0,0.569072,0.999788,0.997743,1.0,0.000000,0.137118,0.336302,0.257737,0.000000,...,0.006392,0.998008,0.757378,0.440348,0.0,0.305263,0.627907,1.0,0.011518,0.006599
41210,1.0,0.571134,0.476162,0.831948,1.0,0.626985,0.668817,0.691373,0.583300,0.507937,...,0.456115,0.997538,0.683084,0.381514,0.0,0.673684,0.864693,1.0,0.478997,0.480833
41211,1.0,0.573196,0.833146,0.999169,1.0,0.060269,0.188905,0.353327,0.262186,0.146592,...,0.133467,0.998395,0.723458,0.424529,0.0,0.357895,0.651163,1.0,0.152439,0.152314
41212,1.0,0.575258,0.476164,0.831354,1.0,0.626985,0.670568,0.676010,0.571856,0.507937,...,0.455282,0.997649,0.678501,0.379915,1.0,0.673684,0.864693,1.0,0.482724,0.483101


cycle - unit of time expressed in machine cycles

s1 - a sensor value measuring machine performance

s2 - another sensor value measuring machine performance

Stage 1: Good performance. s1 is random between (0,100). s2 is a gamma distribution with shape k=0.2.

Stage 2: Degraded performance. s1 is random between (500,750). s2 is a gamma distribution with shape k=5.

Stage 3: Failed performance. s1 is random between (500,1000). s2 continues with as gamma distribution with shape k=5.